# Santa Catarina TFA inversion

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cPickle as pickle
import os
import pandas as pd

from fatiando import utils
from fatiando.gravmag import polyprism
from fatiando.mesher import PolygonalPrism
from fatiando.vis import mpl, myv
from matplotlib import colors, ticker, cm
from IPython.display import Image as img
from matplotlib.mlab import normpdf

/home/leo/anaconda2/lib/python2.7/site-packages/fatiando/vis/mpl.py:76: UserWarning: This module will be removed in v0.6. We recommend the use of matplotlib.pyplot module directly. Some of the fatiando specific functions will remain.
  "specific functions will remain.")
/home/leo/anaconda2/lib/python2.7/site-packages/fatiando/vis/myv.py:51: UserWarning: This module will be removed in v0.7.
  warnings.warn("This module will be removed in v0.7.")


### Auxiliary functions

In [2]:
import sys
sys.path.insert(0, '../../code')

import mag_polyprism_functions as mfun

# Input

### Importing Diorama TFA data

In [3]:
data = pd.read_csv('anitapolis_mag.txt', skipinitialspace=True, delim_whitespace=True)

In [4]:
data['GPSALT'] = - data['GPSALT'] + 800

In [5]:
mask = (data['GPSALT'].get_values()<0.)
data = data[mask]

In [6]:
xp = data['X'].get_values()
yp = data['Y'].get_values()
zp = data['GPSALT'].get_values()
alt = data['ALTURA'].get_values()
dobs = data['mag_res1'].get_values()

### Parameters for inversion

In [7]:
# output of inversion
inversion = dict()

In [8]:
N = xp.size

In [9]:
#initial estimate
M = 20 # number of vertices per prism
L = 6 # number of prisms
P = L*(M+2) + 1 # number of parameters

#figura
incs = -21
decs = -11.
intensity = 15.

#figura
z0 = 0.
dz = 630.
r = 700.
props = {'magnetization': utils.ang2vec(
        intensity, incs, decs)}

rin = np.zeros(M) + r
m0 = np.hstack((rin, np.array([6921000., 688000.])))
m0 = np.resize(m0, P - 1) # inicial parameters vector
m0 = np.hstack((m0, dz))
model0 = mfun.param2polyprism(m0, M, L, z0, props) # list of classes of prisms

# main field
inc = -37.05
dec = -18.17

# predict data
d0 = polyprism.tf(xp, yp, zp, model0, inc, dec)

In [10]:
# limits for parameters in meters
rmin = 10.
rmax = 1200.
y0min = 687000.
y0max = 699000.
x0min = 6916000.
x0max = 6923000.
dzmin = 10.
dzmax = 1000.

mmin, mmax = mfun.build_range_param(M, L, rmin, rmax, x0min, x0max, y0min, y0max, dzmin, dzmax)

In [11]:
# variation for derivatives
deltax = 0.01*np.max(100.)
deltay = 0.01*np.max(100.)
deltar = 0.01*np.max(100.)
deltaz = 0.01*np.max(100.)
delta = np.array([deltax, deltay, deltar, deltaz])

In [12]:
m_out = np.zeros(M + 2)  # outcropping body parameters

### Regularization parameters

In [13]:
#lamb = th*0.01 # Marquadt's parameter
lamb = 10.0
dlamb = 10.      # step for Marquadt's parameter

a1 = 1.0e-4  # adjacent radial distances within each prism
a2 = 1.0e-3   # vertically adjacent radial distances
a3 = 0.     # outcropping cross-section
a4 = 0.     # outcropping origin
a5 = 1.0e-4 # vertically adjacent origins
a6 = 1.0e-8   # zero order Tikhonov on adjacent radial distances
a7 = 1.0e-5     # zero order Tikhonov on thickness of each prism

In [14]:
foldername = 'test'

In [15]:
alpha = np.array([a1, a2, a3, a4, a5, a6, a7])

In [16]:
itmax = 30
itmax_marq = 10
tol = 1.0e-4     # stop criterion

### Inversion

In [17]:
d_fit, m_est, model_est, phi_list, model_list, res_list = mfun.levmarq_tf(xp, yp, zp, m0, M, L, delta, itmax,
                                                    itmax_marq, lamb, dlamb, tol,
                                                    mmin, mmax, m_out, dobs, inc, dec,
                                                    props, alpha, z0, dz)

it:  0   it_marq:  0   lambda: 1e+01   init obj.: 3.31542e+03  fin obj.: 1.33051e+03
it:  1   it_marq:  0   lambda: 1e+00   init obj.: 1.33051e+03  fin obj.: 6.83517e+02
it:  2   it_marq:  0   lambda: 1e-01   init obj.: 6.83517e+02  fin obj.: 8.72722e+02
it:  2   it_marq:  1   lambda: 1e+00   init obj.: 6.83517e+02  fin obj.: 5.24479e+02
it:  3   it_marq:  0   lambda: 1e-01   init obj.: 5.24479e+02  fin obj.: 6.70555e+02
it:  3   it_marq:  1   lambda: 1e+00   init obj.: 5.24479e+02  fin obj.: 4.82526e+02
it:  4   it_marq:  0   lambda: 1e-01   init obj.: 4.82526e+02  fin obj.: 6.40640e+02
it:  4   it_marq:  1   lambda: 1e+00   init obj.: 4.82526e+02  fin obj.: 4.49219e+02
it:  5   it_marq:  0   lambda: 1e-01   init obj.: 4.49219e+02  fin obj.: 6.26722e+02
it:  5   it_marq:  1   lambda: 1e+00   init obj.: 4.49219e+02  fin obj.: 4.29344e+02
it:  6   it_marq:  0   lambda: 1e-01   init obj.: 4.29344e+02  fin obj.: 6.17227e+02
it:  6   it_marq:  1   lambda: 1e+00   init obj.: 4.29344e+02  fi

In [21]:
# output of inversion
inversion = dict()

inversion['xp'] = xp
inversion['yp'] = yp
inversion['zp'] = zp
inversion['observed_data'] = dobs
inversion['inc_dec'] = [incs, decs]
inversion['z0'] = z0
inversion['initial_dz'] = dz
inversion['intial_r'] = r
inversion['initial_estimate'] = model0
inversion['initial_data'] = d0
inversion['limits'] = [rmin, rmax, x0min, x0max, y0min, y0max, dzmin, dzmax]
inversion['regularization'] = np.array([a1, a2, a3, a4, a5, a6, a7])
inversion['tol'] = tol
inversion['main_field'] = [-21.5, -18.7]
inversion['data_fit'] = d_fit
inversion['estimate'] = m_est
inversion['prisms'] = model_est
inversion['estimated_models'] = model_list
inversion['objective'] = phi_list
inversion['residual'] = dobs - d_fit
inversion['residual_list'] = res_list

### Folder to save the results

In [22]:
if foldername == '':
    mypath = 'results/single-'+d4 #default folder name
    if not os.path.isdir(mypath):
       os.makedirs(mypath)
else:
    mypath = 'results/single-'+foldername #defined folder name
    if not os.path.isdir(mypath):
       os.makedirs(mypath)

In [23]:
file_name = mypath+'/inversion.pickle'
with open(file_name, 'w') as f:
    pickle.dump(inversion, f)